<a href="https://colab.research.google.com/github/MatheusBs10/Lab-ITS-IA/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuração com teste da API

In [ ]:
import json
from google.colab import userdata

API_KEY = userdata.get('API_KEY')

if API_KEY is not None:
  print("API_KEY loaded successfully.")
else:
  print("API_KEY not found in environment variables.")
import google.generativeai as genai

genai.configure(api_key=API_KEY)


llm = genai.GenerativeModel('models/gemini-2.5-flash')

print(llm.generate_content("ping").text)
print(llm.generate_content("já falei ping?").text)
chat = llm.start_chat()

resposta = chat.send_message("ping")
print(resposta.text)
resposta2 = chat.send_message("já falei ping?")
print(resposta2.text)

print("---Historico---")
print(chat.history)

Modelo de Domínio

In [ ]:
def etapa_0_prep_modelo_dominio(llm, dominio, n_topicos=5, audiencia="4o ano do Ensino Fundamental"):
    """
    Define o Modelo de Domínio em tópicos, cada um com explicação, pré-requisito, e exercício.
    """
    print(f"Gerando Modelo de Domínio para '{dominio}'...")

    prompt_dominio = f"""
    Você é especializado no domínio `{dominio}`.
    Pense nos {n_topicos} tópicos mais relevantes para a audiência "{audiencia}".
    Para cada tópico, precisará de (1) uma explicação espontaneamente simples,
    (2) a lista dos outros tópicos que são pré-requisitos deste tópico e (3) um
    exercício para avaliar a compreensão do tópico.
    As explicações e exercícios devem ser auto-contidos, não referenciando outras frases.
    O formato da sua saída deverá ser um json válido.
    """
    exemplo_base_conhecimento = {
        "soma_fracoes": {
            "explicacao": "Para somar frações, primeiro encontre um denominador comum. Depois, some os numeradores.",
            "pre_requisitos": ["denominador_comum"],
            "exercicio": "Quanto é 1/2 + 1/4?"
        },
        "denominador_comum": {
            "explicacao": "O denominador comum é um múltiplo compartilhado pelos denominadores de duas ou mais frações.",
            "pre_requisitos":[],
            "exercicio": "Qual é o menor denominador comum para 1/3 e 1/5?"
        },
        "subtracao_fracoes": {
            "explicacao": "A subtração de frações segue a mesma lógica da soma: encontre um denominador comum e depois subtraia os numeradores.",
            "pre_requisitos": ["denominador_comum", "soma_fracoes"],
            "exercicio": "Quanto é 3/4 - 1/4?"
        }
    }

    exemplo_base_conhecimento_json = json.dumps(exemplo_base_conhecimento, indent=4, ensure_ascii=False)

    prompt_dominio += f"\n Exemplo:\n {exemplo_base_conhecimento_json}"

    resposta_ia = llm.generate_content(prompt_dominio).text
    modelo_dominio_json = resposta_ia.strip().lstrip("```json").rstrip("```")

    try:
        modelo_dominio_dict = json.loads(modelo_dominio_json)
        print("Modelo de Domínio gerado e carregado com sucesso!")
        return modelo_dominio_dict
    except json.JSONDecodeError:
        print("--- ERRO AO DECODIFICAR JSON ---")
        print(modelo_dominio_json)
        return None

modelo_dominio = etapa_0_prep_modelo_dominio(llm, "Frações", 5, "4o ano do Ensino Fundamental")

if modelo_dominio:
    print("\n--- Modelo de Domínio (Resultado) ---")
    print(json.dumps(modelo_dominio, indent=2, ensure_ascii=False))

Modelo do Aluno

In [ ]:
def etapa_0_inicializar_aluno(modelo_dominio_dict):
    """
    Inicializando o modelo do aluno com nível 'iniciante' em todos os tópicos.
    """
    if modelo_dominio_dict is None:
        print("Erro: Modelo de domínio não foi carregado. Não é possível inicializar o aluno.")
        return {}


    modelo_aluno = {topico: "iniciante" for topico in modelo_dominio_dict.keys()}
    return modelo_aluno

modelo_aluno = etapa_0_inicializar_aluno(modelo_dominio)
print("--- Modelo do Aluno (Inicial) ---")
print(json.dumps(modelo_aluno, indent=2))

Seleção do próximo tópico

In [ ]:
def etapa_1_selecao_proximo_topico(llm, modelo_aluno, modelo_dominio):
    """
    Usa o LLM para decidir qual a próxima tarefa (Macro-Adaptação).
    """
    print("\nDecidindo próximo tópico...")

    topicos_com_prerequisitos = {}
    for topico, detalhes in modelo_dominio.items():
        topicos_com_prerequisitos[topico] = detalhes.get("pre_requisitos", [])

    prompt_selecao_tarefa = f"""
    Você é um Sistema de Tutoria Inteligente (ITS).
    Decida o próximo tópico a ser ensinado com base no Modelo do Aluno a seguir:

    **Modelo do Aluno (Níveis de Maestria)**:
    {json.dumps(modelo_aluno, indent=4, ensure_ascii=False)}

    **Tópicos Disponíveis e seus Pré-requisitos**:
    {json.dumps(topicos_com_prerequisitos, indent=4, ensure_ascii=False)}

    Pense passo a passo para tomar sua decisão:
    1. Identifique os tópicos que o aluno ainda NÃO aprendeu (nível 'iniciante').
    2. Desses, filtre aqueles cujos pré-requisitos JÁ foram aprendidos (nível 'avançado' ou 'aprendendo'). Se um tópico não tem pré-requisitos, ele é válido.
    3. Se houver vários tópicos válidos, escolha o mais fundamental.
    4. Retorne sua decisão em um formato JSON válido com a chave "proximo_topico" (o nome exato do tópico escolhido) e "raciocinio" (2-3 frases ultra-concisas explicando o porquê).
    """

    resposta_ia = llm.generate_content(prompt_selecao_tarefa).text
    decisao_json = resposta_ia.strip().lstrip("```json").rstrip("```")

    try:
        decisao_dict = json.loads(decisao_json)
        print("Decisão tomada pela IA.")
        return decisao_dict
    except json.JSONDecodeError:
        print("--- ERRO AO DECODIFICAR JSON (Decisão) ---")
        print(decisao_json)
        return None

decisao = etapa_1_selecao_proximo_topico(llm, modelo_aluno, modelo_dominio)

if decisao and 'proximo_topico' in decisao:
    topico_escolhido = decisao['proximo_topico']
    print("\n--- Próximo tópico ---")
    print(json.dumps(decisao, indent=2, ensure_ascii=False))
else:
    print("\n--- Erro na seleção de tópico ---")
    topico_escolhido = None

Avaliação da interação inicial

In [ ]:
def etapa_3_avaliacao_interacao_inicial(llm, chat, modelo_aluno, topico_atual, exercicio):
    """
    Usa o LLM para avaliar a resposta inicial e atualizar o modelo do aluno.
    """

    if not chat.history or chat.history[-1].role != 'user':
        print("Erro: Nenhuma resposta do usuário encontrada no histórico recente.")
        return modelo_aluno

    mensagem_usuario = chat.history[-1].parts[0].text

    prompt_avaliacao = f"""
    Você é um Avaliador de um Sistema de Tutoria Inteligente.
    Seu objetivo é avaliar a primeira resposta do aluno a um exercício e atualizar seu nível de maestria.

    **Tópico Atual:** {topico_atual}
    **Exercício Proposto:** "{exercicio}"
    **Resposta do Aluno:** "{mensagem_usuario}"
    **Nível de Maestria Atual:** {modelo_aluno[topico_atual]}

    Pense passo a passo:
    1. Analise a resposta do aluno. A resposta está correta, parcialmente correta, ou completamente incorreta?
    2. Com base na resposta, decida qual o novo nível de maestria do aluno para este tópico.
       - 'iniciante': Resposta muito errada ou não entendeu a pergunta.
       - 'aprendendo': Tentou, mas cometeu erros conceituais, ou acertou parcialmente.
       - 'avançado': Respondeu corretamente e demonstrou entendimento.

    Retorne sua decisão em um formato JSON válido com as chaves "novo_nivel" e "raciocinio".
    """

    resposta_ia = llm.generate_content(prompt_avaliacao).text
    avaliacao_json = resposta_ia.strip().lstrip("```json").rstrip("```")

    try:
        avaliacao = json.loads(avaliacao_json)
        novo_nivel = avaliacao.get('novo_nivel')

        if novo_nivel in ['iniciante', 'aprendendo', 'avançado']:

            modelo_aluno[topico_atual] = novo_nivel
            print(f"Nível do aluno para '{topico_atual}' atualizado para: {novo_nivel}")
        else:
            print(f"Erro: Nível '{novo_nivel}' recebido da IA não é válido.")

        return modelo_aluno

    except json.JSONDecodeError:
        print("--- ERRO AO DECODIFICAR JSON (Avaliação Etapa 3) ---")
        print(avaliacao_json)
        return modelo_aluno


chat_simulado = llm.start_chat()
chat_simulado.send_message("A resposta para 'Qual é o menor denominador comum para 1/3 e 1/5?' é 15.")


if topico_escolhido:
    exercicio_teste = modelo_dominio[topico_escolhido].get('exercicio', 'Exercício não encontrado')
    modelo_aluno_atualizado = etapa_3_avaliacao_interacao_inicial(llm, chat_simulado, modelo_aluno, topico_escolhido, exercicio_teste)

    print("\n--- Modelo do Aluno (pós-avaliação inicial) ---")
    print(json.dumps(modelo_aluno_atualizado, indent=2))
else:
    print("Teste da Célula 5 pulado (nenhum tópico foi escolhido na Célula 4).")

Decidir e Gerar Feedback

In [ ]:
def etapa_45_decidir_e_gerar_feedback(chat, exercicio):
    """
    Usa o LLM para avaliar a resposta e gerar feedback adaptativo (Micro-Adaptação).
    """
    if not chat.history or chat.history[-1].role != 'user':
        print("Erro: Nenhuma resposta do usuário encontrada para dar feedback.")
        return "Por favor, tente responder o exercício."

    resposta_aluno = chat.history[-1].parts[0].text

    prompt_feedback = f"""
    --- Decisão de feedback ---
    Você é um Tutor de um Sistema de Tutoria Inteligente (ITS).
    Avalie a resposta do aluno e forneça um feedback útil.

    Exercício Proposto: *"{exercicio}"*
    Resposta do Aluno: **"{resposta_aluno}"**

    Pense passo a passo:
    1. A resposta do aluno está correta?
    2. Se estiver correta, elogie e confirme.
    3. Se estiver incorreta, identifique o equívoco principal.
    4. Gere uma dica ou uma pergunta socrática (sem dar a resposta direta) para ajudar o aluno a encontrar o caminho certo.
    5. Formule sua resposta final para o aluno de forma encorajadora.
    ----------------------------
    Feedback:
    """

    resposta_feedback = llm.generate_content(prompt_feedback).text


    chat.send_message(resposta_feedback)

    return resposta_feedback

chat_simulado_2 = llm.start_chat()
chat_simulado_2.send_message("A resposta para 'Qual é o menor denominador comum para 1/3 e 1/5?' é 8.")

if topico_escolhido:
    exercicio_teste_2 = modelo_dominio[topico_escolhido].get('exercicio', 'Exercício não encontrado')
    feedback = etapa_45_decidir_e_gerar_feedback(chat_simulado_2, exercicio=exercicio_teste_2)

    print("\n--- Feedback Gerado (Etapa 4/5) ---")
    print(feedback)
else:
     print("Teste da Célula 6 pulado (nenhum tópico foi escolhido na Célula 4).")

Atualização Pós-Feedback

In [ ]:
def etapa_7_atualizacao_pos_feedback(llm, chat, modelo_aluno, topico_atual):
    """
    Atualiza o modelo do aluno com base no último ciclo de conversa (pós-feedback).
    """


    if len(chat.history) > 3:

        ciclo_interacao = chat.history[-4:]


        historico_formatado = "\n".join([f"{msg.role}: {msg.parts[0].text}" for msg in ciclo_interacao])

        prompt_reavaliacao = f"""
        Você é um Avaliador de um Sistema de Tutoria Inteligente.
        Reavalie o nível de maestria do aluno após um ciclo de feedback.

        **Tópico Atual:** {topico_atual}
        **Nível de Maestria Atual:** {modelo_aluno[topico_atual]}

        **Ciclo de Interação Recente:**
        {historico_formatado}

        Pense passo a passo:
        1. O aluno recebeu feedback (dica) na penúltima mensagem.
        2. A última mensagem do aluno ("user:") mostra que ele corrigiu o erro?
        3. Com base nesta última tentativa, atualize o nível de maestria.
           - 'aprendendo': Se ainda comete erros, mas melhorou.
           - 'avançado': Se corrigiu o erro e acertou.

        Retorne sua decisão em um formato JSON válido com as chaves "novo_nivel" e "raciocinio".
        """

        resposta_ia = llm.generate_content(prompt_reavaliacao).text
        avaliacao_json = resposta_ia.strip().lstrip("```json").rstrip("```")

        try:
            avaliacao = json.loads(avaliacao_json)
            novo_nivel = avaliacao.get('novo_nivel')

            if novo_nivel in ['iniciante', 'aprendendo', 'avançado']:
                modelo_aluno[topico_atual] = novo_nivel
                print(f"Nível (pós-feedback) para '{topico_atual}' atualizado para: {novo_nivel}")
            else:
                print(f"Erro: Nível '{novo_nivel}' (Etapa 7) recebido da IA não é válido.")

            return modelo_aluno

        except json.JSONDecodeError:
            print("--- ERRO AO DECODIFICAR JSON (Avaliação Etapa 7) ---")
            print(avaliacao_json)
            return modelo_aluno

    else:

        return modelo_aluno

chat_simulado_2.send_message("Ah, entendi! É o produto deles, 15.")

if topico_escolhido:
    modelo_aluno_final = etapa_7_atualizacao_pos_feedback(llm, chat_simulado_2, modelo_aluno, topico_escolhido)
    print("\n--- Modelo do Aluno (pós-feedback) ---")
    print(json.dumps(modelo_aluno_final, indent=2))
else:
     print("Teste da Célula 7 pulado (nenhum tópico foi escolhido na Célula 4).")

Função principal

In [ ]:
def sistema_tutoria_inteligente_genai(llm, dominio, n_topicos=5, audiencia="4o ano do Ensino Fundamental"):
    """
    Inicia uma nova sessão de tutoria completa.
    """


    prompt_sistema = {"role": "model", "parts": ["Você é um Sistema de Tutoria Inteligente (ITS). Seu objetivo é ensinar, dar exercícios e feedback adaptativo. Seja encorajador e claro."]}

    chat_principal = llm.start_chat(history=[prompt_sistema])


    modelo_dominio = etapa_0_prep_modelo_dominio(llm, dominio, n_topicos, audiencia)
    if not modelo_dominio:
        print("Falha ao criar o Modelo de Domínio. Encerrando.")
        return


    modelo_aluno = etapa_0_inicializar_aluno(modelo_dominio)


    topicos_restantes = list(modelo_aluno.keys())

    print("\n" + "="*50)
    print(f"🚀 INICIANDO TUTORIA SOBRE: {dominio} 🚀")
    print("="*50)


    while any(nivel != 'avançado' for nivel in modelo_aluno.values()):


        decisao_json = etapa_1_selecao_proximo_topico(llm, modelo_aluno, modelo_dominio)

        if not decisao_json or 'proximo_topico' not in decisao_json:
            print("IA não conseguiu decidir o próximo tópico. Verificando se terminamos...")

            if any(nivel != 'avançado' for nivel in modelo_aluno.values()):
                 print("Erro: A IA falhou em selecionar um tópico, mas ainda há tópicos pendentes.")
            break

        topico = decisao_json['proximo_topico']
        print(f"\n--- Próximo Tópico: {topico} ---")
        print(f"(Raciocínio da IA: {decisao_json.get('raciocinio', 'N/A')})")


        explicacao = modelo_dominio[topico]['explicacao']
        exercicio = modelo_dominio[topico]['exercicio']


        prompt_para_aluno = f"{explicacao}\n\nVamos praticar:\n{exercicio}"
        chat_principal.send_message(prompt_para_aluno)
        print(f"\n[TUTOR]: {prompt_para_aluno}")


        mensagem_usuario = input("[VOCÊ]: ")
        chat_principal.send_message(mensagem_usuario)
        modelo_aluno = etapa_3_avaliacao_interacao_inicial(llm, chat_principal, modelo_aluno, topico, exercicio)

        while modelo_aluno.get(topico) != 'avançado':
            print(f"Nível atual: {modelo_aluno.get(topico)}. Iniciando ciclo de feedback.")

            feedback = etapa_45_decidir_e_gerar_feedback(chat_principal, exercicio)
            print(f"\n[TUTOR] (Feedback): {feedback}")

            mensagem_usuario_feedback = input("[VOCÊ] (Após a dica): ")
            chat_principal.send_message(mensagem_usuario_feedback)

            modelo_aluno = etapa_7_atualizacao_pos_feedback(llm, chat_principal, modelo_aluno, topico)

        print(f"\n🎉 Ótimo! Tópico '{topico}' concluído com nível 'avançado'!")

    print("\n" + "="*50)
    print("🎓 PARABÉNS! Você concluiu todos os tópicos! 🎓")
    print("="*50)

    return chat_principal.history

Executar sistema completo

In [ ]:
dominio_desejado = "Regras básicas de Xadrez"
audiencia_desejada = "Iniciantes (Adultos)"


historico_final = sistema_tutoria_inteligente_genai(
    llm,
    dominio=dominio_desejado,
    n_topicos=3,
    audiencia=audiencia_desejada
)

Nome: << MATHEUS BESERRA SOUZA >>

1. Avalie a eficácia do seu ITS para apoiar a aprendizagem de três tipos de alunos: o aluno iniciante, o intermediário e o avançado.
RESPOSTA:
  Estudante iniciante
 O sistema se destaca para os iniciantes.

 Ele realiza o scaffolding (andaime) de maneira adequada, assegurando que a pessoa aprenda os pré-requisitos antes de prosseguir (a Etapa 1 funciona).

 O loop de micro-adaptação (Etapas 4-7) é muito bom: ele não permite que o estudante avance se cometer um erro, oferece uma dica e exige uma nova tentativa.  É ideal para iniciantes que estão aprendendo do zero.

   Estudante de nível intermediário
 Neste ponto, o sistema começa a apresentar falhas.

 A etapa_0_inicializar_aluno é o principal problema.  O sistema obriga todos a começar como 'iniciante' em todas as áreas.

 O estudante intermediário, que já possui o conhecimento básico, ficará entediado.  Ele precisará realizar uma série de exercícios simples apenas para demonstrar ao sistema que possui conhecimento.

 O que está pendente: realizar um teste de nivelamento (pré-avaliação) antes de iniciar o ensino.

   Aluno Avançado
 Esse aluno considera o sistema ineficiente e frustrante.

 É o mesmo problema do intermediário, só que mais grave.  Ele terá que percorrer todo o currículo (que já domina) apenas por formalidade.

 O sistema não possui a capacidade de "aprofundar" o conteúdo.  Quando o estudante demonstra conhecimento dos cinco tópicos daquele domínio, o sistema encerra.  É um limite de conhecimento muito baixo.

 Resumo: O ITS é excelente para alfabetizar iniciantes, porém é bastante tedioso para aqueles que já possuem algum conhecimento (nível intermediário) e irrelevante para quem deseja se aprofundar (nível avançado).

2. Faça uma simulação de tutoria fictícia de um aluno iniciante (interpretado por você) nos domínios de fotossíntese e coesão textual, quais são os prós e os contras dessa abordagem de tutoria?

Simulações (O que ocorre)
 Fotossíntese: O tutor me ensina e questiona sobre os três componentes.  Eu me esqueço do "dióxido de carbono".  O tutor não fornece a resposta, mas oferece uma pista: "o que a planta respira".  Na segunda tentativa, eu acerto, e ele me classifica como 'avançado' no assunto.

 Coesão Textual: O tutor ensina "porém" (oposição) e solicita que se complete uma frase.  Eu utilizo "portanto" para indicar conclusão.  O sistema identifica que eu confundi o conceito, fornece um feedback esclarecendo a diferença, e eu acerto na segunda tentativa.

Vantagens (O que o sistema faz bem)
 Bom para iniciantes: ele assegura que você aprenda os pré-requisitos antes de prosseguir (Etapa 1).  Trata-se de um bom scaffolding.

 Feedback imediato: O loop de micro-adaptação (Etapas 4-7) é o destaque.  Você não avança se errar; ele te obriga a tentar novamente com uma dica.

 Flexível: a Etapa 0, responsável por criar o currículo (Domínio), é bastante versátil.  Posso solicitar que ele desenvolva um curso de "Xadrez" ou "Frações".

Desvantagens (Onde o sistema não funciona)
 Chato para quem não é iniciante: a Etapa 0 (Inicialização) obriga todos a começar do começo.  Se você já domina o básico, terá que fazer exercícios simples apenas para comprovar, o que é entediante.

 Frágil (Depende do JSON): se a IA cometer um erro de pontuação no JSON, todo o sistema falha.  Trata-se de um ponto de falha bastante arriscado.

 Linear e "Burro": O tutor não "conversa".  Se eu fizer uma pergunta que não está no script (por exemplo, "Por que a clorofila é verde?"), é provável que ele fique confuso, já que só está preparado para avaliar as respostas do exercício.

3. Quais seriam bons próximos passos para aprimorar essa implementação de tutoria?

Implementar uma Pré-Avaliação (Nivelamento)
Atualmente, o maior gargalo do sistema é a etapa_0_inicializar_aluno, que força todo mundo a começar do zero.

O que fazer: Criar uma "Etapa -1: Teste de Nivelamento".

Como: Antes de iniciar o ensino, o sistema aplicaria um "quiz" rápido com 1 pergunta de cada tópico do modelo_dominio. Se o aluno acertar, o modelo_aluno já começaria com aquele tópico marcado como 'avançado'.

Resultado: Isso resolve 100% o problema do tédio para os alunos intermediários e avançados, permitindo que o tutor pule direto para onde o aluno realmente precisa de ajuda.

Criar um "Roteador de Intenção" (Para Sair do Linear)
O sistema atual é "burro" e linear. Ele só sabe avaliar se a resposta do exercício está certa. Se o aluno fizer uma pergunta ou expressar frustração, o sistema avalia isso como uma "resposta errada".

O que fazer: Criar um "roteador" que primeiro entenda a intenção do aluno antes de agir.

Como: Antes da etapa_3_avaliacao_interacao_inicial, rodamos um prompt de classificação simples:

A resposta do aluno é uma "Resposta ao Exercício"? (Ex: "A resposta é 15.") -> Se sim, continue para a Etapa 3.

A resposta é uma "Pergunta sobre o Tópico"? (Ex: "Por que a clorofila é verde?") -> Se sim, chame uma nova função responder_pergunta() e depois repita o exercício.

A resposta é uma "Frustração/Confusão"? (Ex: "Não entendi nada.") -> Se sim, chame uma nova função reexplicar_topico() (que pede à IA uma analogia diferente).

Resultado: O tutor deixa de ser um "avaliador de formulário" e começa a agir como um tutor de verdade, capaz de lidar com conversas que saem do script.